In [2]:
import joblib
import pandas as pd
import numpy as np
from collections import defaultdict
import os

## Cargar Datos

In [3]:
train_data = pd.read_csv('../Datos/validacion_historica/transacciones_entrenamiento.csv', encoding='utf-8')
test_data = pd.read_csv('../Datos/validacion_historica/transacciones_validacion.csv', encoding='utf-8')


## Analisis de Overlap

In [4]:
clientes_entrenamiento = set(train_data['id'].unique())
clientes_validacion = set(test_data['id'].unique())
clientes_comunes = clientes_entrenamiento.intersection(clientes_validacion)

productos_entrenamiento = set(train_data['producto'].unique())
productos_validacion = set(test_data['producto'].unique())
productos_comunes = productos_entrenamiento.intersection(productos_validacion)

print(f"Clientes comunes: {len(clientes_comunes):,} / {len(clientes_entrenamiento):,} ({len(clientes_comunes)/len(clientes_entrenamiento)*100:.1f}%)")
print(f"Productos comunes: {len(productos_comunes):,} / {len(productos_entrenamiento):,} ({len(productos_comunes)/len(productos_entrenamiento)*100:.1f}%)")


Clientes comunes: 11,330 / 404,601 (2.8%)
Productos comunes: 3,085 / 7,177 (43.0%)


## Importar modelo

In [5]:
# Ruta del modelo exportado
MODEL_PATH = "../Modelos/ModelosTrain/historical_model_train.pkl"

# Cargar el modelo
try:
    modelo_historico_importado = joblib.load(MODEL_PATH)
    print("✅ Modelo histórico cargado exitosamente desde archivo PKL!")
    
    # Mostrar información del modelo
    print(f"\n📊 INFORMACIÓN DEL MODELO IMPORTADO:")
    print(f"   - Tipo: {modelo_historico_importado.get('model_type', 'N/A')}")
    print(f"   - Creado desde: {modelo_historico_importado.get('created_from', 'N/A')}")
    print(f"   - Usuarios: {modelo_historico_importado['n_users']:,}")
    print(f"   - Productos: {modelo_historico_importado['n_items']:,}")
    print(f"   - Interacciones: {modelo_historico_importado['user_item_matrix_csr'].nnz:,}")
    print(f"   - Estado: {'✅ Listo' if modelo_historico_importado['data_ready'] else '❌ No listo'}")
    
except FileNotFoundError:
    print(f"❌ Error: No se encontró el archivo {MODEL_PATH}")
    print("💡 Asegúrate de haber ejecutado la celda de exportación primero")
    modelo_historico_importado = None
except Exception as e:
    print(f"❌ Error cargando modelo: {e}")
    modelo_historico_importado = None

✅ Modelo histórico cargado exitosamente desde archivo PKL!

📊 INFORMACIÓN DEL MODELO IMPORTADO:
   - Tipo: historical_collaborative_filtering
   - Creado desde: algoritmo_de_recomendacion_B2C_historial.ipynb
   - Usuarios: 404,601
   - Productos: 7,177
   - Interacciones: 1,712,849
   - Estado: ✅ Listo


## FUNCIÓN PARA USAR EL MODELO IMPORTADO


In [6]:

def recomendar_con_modelo_importado(cliente_id, N=10, k_similar_items=30):
    """
    Genera recomendaciones usando el modelo histórico importado desde PKL
    
    Args:
        cliente_id (int): ID del cliente
        N (int): Número de recomendaciones
        k_similar_items (int): Productos similares a considerar
    
    Returns:
        pandas.DataFrame: Recomendaciones
    """
    if modelo_historico_importado is None:
        print("❌ Modelo no disponible")
        return pd.DataFrame()
    
    if not modelo_historico_importado['data_ready']:
        print("❌ Modelo no está listo")
        return pd.DataFrame()
    
    # Extraer componentes del modelo
    user_item_matrix_csr = modelo_historico_importado['user_item_matrix_csr']
    item_similarity_matrix = modelo_historico_importado['item_similarity_matrix']
    user_to_idx = modelo_historico_importado['user_to_idx']
    idx_to_item = modelo_historico_importado['idx_to_item']
    
    # Verificar que el cliente existe
    if cliente_id not in user_to_idx:
        print(f"❌ Cliente {cliente_id} no encontrado en el modelo")
        return pd.DataFrame()
    
    try:
        # Obtener historial del cliente
        target_user_idx = user_to_idx[cliente_id]
        user_purchased_items_indices = user_item_matrix_csr[target_user_idx].indices
        
        if len(user_purchased_items_indices) == 0:
            print(f"⚠️ Cliente {cliente_id} no tiene historial de compras")
            return pd.DataFrame()
        
        print(f"👤 Cliente {cliente_id} compró {len(user_purchased_items_indices)} productos")
        
        # Acumular puntajes de candidatos
        candidate_items = defaultdict(float)
        
        for purchased_item_idx in user_purchased_items_indices:
            if purchased_item_idx >= item_similarity_matrix.shape[0]:
                continue
            
            # Obtener similitudes
            item_similarities_vector = item_similarity_matrix[purchased_item_idx]
            similar_item_indices_sorted = np.argsort(item_similarities_vector)[::-1]
            top_k_indices = similar_item_indices_sorted[:k_similar_items]
            
            # Acumular puntajes
            for similar_item_idx in top_k_indices:
                similarity_score = item_similarities_vector[similar_item_idx]
                if similarity_score > 0:
                    candidate_items[similar_item_idx] += similarity_score
        
        if not candidate_items:
            print(f"⚠️ No se encontraron productos similares para cliente {cliente_id}")
            return pd.DataFrame()
        
        # Crear ranking
        ranked_candidates = []
        for item_idx, score in candidate_items.items():
            item_id = idx_to_item.get(item_idx)
            if item_id:
                ranked_candidates.append({
                    'producto': item_id, 
                    'recommendation_score': float(score)
                })
        
        if not ranked_candidates:
            return pd.DataFrame()
        
        # Ordenar y tomar top N
        recommendations = pd.DataFrame(ranked_candidates)
        recommendations = recommendations.sort_values('recommendation_score', ascending=False).head(N)
        
        print(f"✅ Se encontraron {len(candidate_items)} productos candidatos")
        
        return recommendations
        
    except Exception as e:
        print(f"❌ Error generando recomendaciones: {e}")
        return pd.DataFrame()

## EJEMPLO DE USO DEL MODELO IMPORTADO

In [7]:

def ejemplo_uso_modelo_pkl(cliente_ejemplo = 10):
    """
    Ejemplo completo de uso del modelo importado desde PKL
    """

    
    if modelo_historico_importado is None:
        print("❌ Modelo no disponible para el ejemplo")
        return
    
    print(f"\n👤 PROBANDO CON CLIENTE: {cliente_ejemplo}")
    
    # Generar recomendaciones
    recomendaciones = recomendar_con_modelo_importado(
        cliente_id=cliente_ejemplo, 
        N=10, 
        k_similar_items=30
    )
    
    if not recomendaciones.empty:
        print(f"\n📋 RECOMENDACIONES GENERADAS:")
        print(recomendaciones)
        
        print(f"\n📊 ESTADÍSTICAS:")
        print(f"   - Total recomendaciones: {len(recomendaciones)}")
        print(f"   - Puntaje máximo: {recomendaciones['recommendation_score'].max():.4f}")
        print(f"   - Puntaje mínimo: {recomendaciones['recommendation_score'].min():.4f}")
        print(f"   - Puntaje promedio: {recomendaciones['recommendation_score'].mean():.4f}")
        
        # Mostrar top 3 productos recomendados
        top_3 = recomendaciones.head(3)['producto'].tolist()
        print(f"   - Top 3 productos: {top_3}")
        
    else:
        print("⚠️ No se generaron recomendaciones")
        
        # Sugerir otros clientes para probar
        clientes_disponibles = list(modelo_historico_importado['user_to_idx'].keys())[:10]
        print(f"\n💡 Prueba con otros clientes disponibles: {clientes_disponibles}")

# Ejecutar ejemplo
ejemplo_uso_modelo_pkl(1)


👤 PROBANDO CON CLIENTE: 1
👤 Cliente 1 compró 40 productos
✅ Se encontraron 470 productos candidatos

📋 RECOMENDACIONES GENERADAS:
          producto  recommendation_score
30   producto_1309              2.178540
31   producto_1207              2.173712
2    producto_1209              1.740635
53    producto_141              1.287753
33   producto_1199              1.248537
139    producto_28              1.199980
133    producto_27              1.148925
32    producto_473              1.141466
103   producto_194              1.021391
18   producto_1192              0.964631

📊 ESTADÍSTICAS:
   - Total recomendaciones: 10
   - Puntaje máximo: 2.1785
   - Puntaje mínimo: 0.9646
   - Puntaje promedio: 1.4106
   - Top 3 productos: ['producto_1309', 'producto_1207', 'producto_1209']


## Preparar datos de validación para algoritmo historico

In [8]:
# ================================================================
# PREPARAR DATOS DE VALIDACIÓN PARA ALGORITMO HISTÓRICO
# ================================================================

print("PREPARANDO DATOS DE VALIDACIÓN PARA ALGORITMO HISTÓRICO")
print("="*60)

# Crear diccionario de compras reales por cliente en período de validación
compras_reales_por_cliente = defaultdict(set)
for _, row in test_data.iterrows():
    compras_reales_por_cliente[row['id']].add(row['producto'])

print(f"Clientes únicos en validación: {len(compras_reales_por_cliente):,}")

# Filtrar solo clientes que estaban en entrenamiento (tienen historial en el modelo)
clientes_con_historial = []
clientes_sin_historial = 0

for cliente_id in compras_reales_por_cliente.keys():
    if cliente_id in modelo_historico_importado['user_to_idx']:
        clientes_con_historial.append(cliente_id)
    else:
        clientes_sin_historial += 1

print(f"Clientes con historial en modelo: {len(clientes_con_historial):,}")
print(f"Clientes sin historial en modelo: {clientes_sin_historial:,}")
print(f"Tasa de cobertura: {len(clientes_con_historial)/len(compras_reales_por_cliente)*100:.1f}%")

# Análisis del tamaño de las compras en validación
tamaños_compras = [len(productos) for productos in compras_reales_por_cliente.values()]
print(f"\nEstadísticas de compras en validación:")
print(f"   - Promedio productos por cliente: {np.mean(tamaños_compras):.2f}")
print(f"   - Mediana: {np.median(tamaños_compras):.0f}")
print(f"   - Máximo: {max(tamaños_compras)}")
print(f"   - Mínimo: {min(tamaños_compras)}")

PREPARANDO DATOS DE VALIDACIÓN PARA ALGORITMO HISTÓRICO
Clientes únicos en validación: 25,950
Clientes con historial en modelo: 11,330
Clientes sin historial en modelo: 14,620
Tasa de cobertura: 43.7%

Estadísticas de compras en validación:
   - Promedio productos por cliente: 3.00
   - Mediana: 2
   - Máximo: 124
   - Mínimo: 1


In [9]:
# ================================================================
# PREPARAR DATOS DE INGRESOS PARA ANÁLISIS ECONÓMICO
# ================================================================

print("PREPARANDO DATOS DE INGRESOS")
print("="*40)

# Crear diccionario de ingresos por cliente y producto en período de validación
ingresos_por_cliente_producto = defaultdict(dict)
ingresos_totales_por_cliente = defaultdict(float)

for _, row in test_data.iterrows():
    cliente_id = row['id']
    producto = row['producto']
    precio = row['precio']
    cantidad = row['cantidad']
    ingreso_transaccion = precio * cantidad
    
    # Guardar ingreso por producto (sumando si hay múltiples compras del mismo producto)
    if producto in ingresos_por_cliente_producto[cliente_id]:
        ingresos_por_cliente_producto[cliente_id][producto] += ingreso_transaccion
    else:
        ingresos_por_cliente_producto[cliente_id][producto] = ingreso_transaccion
    
    # Sumar al total del cliente
    ingresos_totales_por_cliente[cliente_id] += ingreso_transaccion

# Estadísticas de ingresos
ingresos_clientes = list(ingresos_totales_por_cliente.values())
print(f"Ingresos en período de validación:")
print(f"   - Total general: ${sum(ingresos_clientes):,.2f}")
print(f"   - Promedio por cliente: ${np.mean(ingresos_clientes):.2f}")
print(f"   - Mediana por cliente: ${np.median(ingresos_clientes):.2f}")
print(f"   - Máximo por cliente: ${max(ingresos_clientes):,.2f}")
print(f"   - Mínimo por cliente: ${min(ingresos_clientes):.2f}")

# Crear diccionario de precios promedio por producto para recomendaciones
precios_promedio_productos = defaultdict(list)
for _, row in test_data.iterrows():
    precios_promedio_productos[row['producto']].append(row['precio'])

precio_promedio_por_producto = {}
for producto, precios in precios_promedio_productos.items():
    precio_promedio_por_producto[producto] = np.mean(precios)

print(f"Productos únicos con precios: {len(precio_promedio_por_producto):,}")

PREPARANDO DATOS DE INGRESOS
Ingresos en período de validación:
   - Total general: $2,965,130.49
   - Promedio por cliente: $114.26
   - Mediana por cliente: $35.46
   - Máximo por cliente: $47,778.59
   - Mínimo por cliente: $0.07
Productos únicos con precios: 3,184


## FUNCIÓN DE EVALUACIÓN DEL ALGORITMO HISTÓRICO


In [10]:
def evaluar_algoritmo_historico_con_ingresos(clientes_evaluar, n_clientes_evaluar=1000, k=10):
    """
    Evalúa el algoritmo histórico con validación temporal incluyendo análisis de ingresos
    
    Args:
        clientes_evaluar: Lista de clientes para evaluar
        n_clientes_evaluar: Número máximo de clientes a evaluar
        k: Top-K recomendaciones a considerar
    
    Returns:
        dict: Métricas acumuladas incluyendo ingresos
    """
    print(f"Clientes a evaluar: {min(n_clientes_evaluar, len(clientes_evaluar))}")
    print(f"Top-K recomendaciones: {k}")
    
    # Tomar muestra de clientes
    clientes_muestra = clientes_evaluar[:n_clientes_evaluar]
    
    # Métricas acumuladas tradicionales
    hit_rates = []
    precisions = []
    recalls = []
    f1_scores = []
    productos_recomendados_total = []
    productos_comprados_total = []
    
    # NUEVAS: Métricas de ingresos
    ingresos_acertados = []           # Dinero de productos acertados
    ingresos_totales_cliente = []     # Dinero total que gastó cada cliente
    ingresos_potenciales_recomendados = []  # Dinero potencial si acepta todas las recomendaciones
    tasa_captura_ingresos = []        # % de ingresos del cliente que capturamos
    
    clientes_evaluados = 0
    clientes_sin_recomendaciones = 0
    errores = 0
    
    for i, cliente_id in enumerate(clientes_muestra):
        # Progreso cada 10%
        if i % max(1, len(clientes_muestra) // 10) == 0:
            print(f"Progreso: {i+1}/{len(clientes_muestra)} ({(i+1)/len(clientes_muestra)*100:.1f}%)")
        
        try:
            # Generar recomendaciones (sin mostrar output)
            import io
            import sys
            from contextlib import redirect_stdout
            
            with redirect_stdout(io.StringIO()):
                recomendaciones = recomendar_con_modelo_importado(cliente_id, N=k, k_similar_items=30)
            
            if recomendaciones.empty:
                clientes_sin_recomendaciones += 1
                continue
            
            # Productos recomendados
            productos_recomendados = set(recomendaciones['producto'].tolist())
            
            # Productos que realmente compró en validación
            productos_comprados_validacion = compras_reales_por_cliente[cliente_id]
            
            # Calcular intersección (aciertos)
            productos_acertados = productos_recomendados.intersection(productos_comprados_validacion)
            
            # MÉTRICAS TRADICIONALES
            hit_rate = 1 if len(productos_acertados) > 0 else 0
            
            if len(productos_recomendados) > len(productos_comprados_validacion):
                precision = len(productos_acertados) / len(productos_comprados_validacion)
            else:
                precision = len(productos_acertados) / len(productos_recomendados)
                
            recall = len(productos_acertados) / len(productos_comprados_validacion) if productos_comprados_validacion else 0
            f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
            
            # NUEVAS MÉTRICAS DE INGRESOS
            
            # Ingresos de productos acertados (lo que realmente ganamos por las recomendaciones)
            ingreso_acertado = 0
            for producto in productos_acertados:
                if producto in ingresos_por_cliente_producto[cliente_id]:
                    ingreso_acertado += ingresos_por_cliente_producto[cliente_id][producto]
            
            # Ingresos totales del cliente en validación
            ingreso_total_cliente = ingresos_totales_por_cliente[cliente_id]
            
            # Ingresos potenciales si acepta todas las recomendaciones (estimación)
            ingreso_potencial_recomendaciones = 0
            for producto in productos_recomendados:
                if producto in precio_promedio_por_producto:
                    # Estimar con precio promedio y cantidad promedio (1 unidad)
                    ingreso_potencial_recomendaciones += precio_promedio_por_producto[producto]
            
            # Tasa de captura de ingresos (qué % del gasto del cliente capturamos con recomendaciones)
            tasa_captura = (ingreso_acertado / ingreso_total_cliente) * 100 if ingreso_total_cliente > 0 else 0
            
            # Guardar métricas tradicionales
            hit_rates.append(hit_rate)
            precisions.append(precision)
            recalls.append(recall)
            f1_scores.append(f1_score)
            productos_recomendados_total.append(len(productos_recomendados))
            productos_comprados_total.append(len(productos_comprados_validacion))
            
            # Guardar métricas de ingresos
            ingresos_acertados.append(ingreso_acertado)
            ingresos_totales_cliente.append(ingreso_total_cliente)
            ingresos_potenciales_recomendados.append(ingreso_potencial_recomendaciones)
            tasa_captura_ingresos.append(tasa_captura)
            
            clientes_evaluados += 1
            
        except Exception as e:
            errores += 1
            if errores <= 5:
                print(f"Error evaluando cliente {cliente_id}: {e}")
            continue
    
    print(f"\nESTADÍSTICAS FINALES:")
    print(f"  Clientes evaluados: {clientes_evaluados}")
    print(f"  Sin recomendaciones: {clientes_sin_recomendaciones}")
    print(f"  Errores: {errores}")
    
    return {
        # Métricas tradicionales
        'hit_rates': hit_rates,
        'precisions': precisions,
        'recalls': recalls,
        'f1_scores': f1_scores,
        'productos_recomendados': productos_recomendados_total,
        'productos_comprados': productos_comprados_total,
        'n_evaluados': clientes_evaluados,
        
        # NUEVAS: Métricas de ingresos
        'ingresos_acertados': ingresos_acertados,
        'ingresos_totales_cliente': ingresos_totales_cliente,
        'ingresos_potenciales_recomendados': ingresos_potenciales_recomendados,
        'tasa_captura_ingresos': tasa_captura_ingresos
    }

In [11]:
print("Esto puede tomar varios minutos...")

# Evaluar con una muestra representativa de clientes
n_clientes_evaluar = len(clientes_con_historial)
k_recomendaciones = 10

resultados = evaluar_algoritmo_historico_con_ingresos(
    clientes_con_historial, 
    n_clientes_evaluar=n_clientes_evaluar,
    k=k_recomendaciones
)

print(f"\nEvaluación completada!")
print(f"Total evaluaciones realizadas: {resultados['n_evaluados']}")

Esto puede tomar varios minutos...
Clientes a evaluar: 11330
Top-K recomendaciones: 10
Progreso: 1/11330 (0.0%)
Progreso: 1134/11330 (10.0%)
Progreso: 2267/11330 (20.0%)
Progreso: 3400/11330 (30.0%)
Progreso: 4533/11330 (40.0%)
Progreso: 5666/11330 (50.0%)
Progreso: 6799/11330 (60.0%)
Progreso: 7932/11330 (70.0%)
Progreso: 9065/11330 (80.0%)
Progreso: 10198/11330 (90.0%)

ESTADÍSTICAS FINALES:
  Clientes evaluados: 11327
  Sin recomendaciones: 3
  Errores: 0

Evaluación completada!
Total evaluaciones realizadas: 11327


In [12]:
if resultados['n_evaluados'] > 0:
    # Calcular métricas promedio tradicionales
    hit_rate_avg = np.mean(resultados['hit_rates'])
    precision_avg = np.mean(resultados['precisions'])
    recall_avg = np.mean(resultados['recalls'])
    f1_avg = np.mean(resultados['f1_scores'])
    
    # NUEVAS: Calcular métricas de ingresos
    ingreso_acertado_promedio = np.mean(resultados['ingresos_acertados'])
    ingreso_total_promedio = np.mean(resultados['ingresos_totales_cliente'])
    ingreso_potencial_promedio = np.mean(resultados['ingresos_potenciales_recomendados'])
    tasa_captura_promedio = np.mean(resultados['tasa_captura_ingresos'])
    
    # Ingresos agregados
    ingreso_total_acertado = sum(resultados['ingresos_acertados'])
    ingreso_total_clientes = sum(resultados['ingresos_totales_cliente'])
    ingreso_total_potencial = sum(resultados['ingresos_potenciales_recomendados'])
    
    
    print(f"\nMÉTRICAS TRADICIONALES:")
    print(f"  Hit Rate@{k_recomendaciones}:     {hit_rate_avg:.3f} ({hit_rate_avg*100:.1f}% clientes con al menos 1 acierto)")
    print(f"  Precision@{k_recomendaciones}:    {precision_avg:.3f}")
    print(f"  Recall@{k_recomendaciones}:       {recall_avg:.3f}")
    print(f"  F1-Score:        {f1_avg:.3f}")
    print(f"  Clientes evaluados: {resultados['n_evaluados']}")
    
    print(f"\n💰 MÉTRICAS DE INGRESOS:")
    print(f"  Ingreso promedio capturado por cliente: ${ingreso_acertado_promedio:.2f}")
    print(f"  Ingreso promedio total por cliente: ${ingreso_total_promedio:.2f}")
    print(f"  Tasa promedio de captura de ingresos: {tasa_captura_promedio:.2f}%")
    print(f"  Ingreso potencial promedio por recomendaciones: ${ingreso_potencial_promedio:.2f}")
    
    print(f"\n💵 INGRESOS AGREGADOS:")
    print(f"  Total capturado por recomendaciones: ${ingreso_total_acertado:,.2f}")
    print(f"  Total gastado por clientes evaluados: ${ingreso_total_clientes:,.2f}")
    print(f"  Total potencial de recomendaciones: ${ingreso_total_potencial:,.2f}")
    print(f"  % del gasto total capturado: {(ingreso_total_acertado/ingreso_total_clientes)*100:.2f}%")
    
    print(f"\n📊 ANÁLISIS DE ROI DEL ALGORITMO:")
    uplift_monetario = (ingreso_total_acertado / ingreso_total_clientes) * 100
    print(f"  Uplift monetario directo: {uplift_monetario:.2f}%")
    print(f"  Ingresos adicionales por cada $100 gastados: ${uplift_monetario:.2f}")
    
    # Proyección anual
    if resultados['n_evaluados'] > 0:
        ingreso_promedio_mensual = ingreso_total_acertado / resultados['n_evaluados']
        proyeccion_anual = ingreso_promedio_mensual * 12
        print(f"  Proyección de ingresos adicionales anuales por cliente: ${proyeccion_anual:.2f}")
    
    # Distribución de hit rates
    hit_rate_distribution = np.bincount(resultados['hit_rates'])
    print(f"\nDISTRIBUCIÓN DE RESULTADOS:")
    print(f"  Clientes sin aciertos: {hit_rate_distribution[0]} ({hit_rate_distribution[0]/resultados['n_evaluados']*100:.1f}%)")
    if len(hit_rate_distribution) > 1:
        print(f"  Clientes con aciertos: {hit_rate_distribution[1]} ({hit_rate_distribution[1]/resultados['n_evaluados']*100:.1f}%)")
    

    metricas_df = pd.DataFrame({
        'algoritmo_historico': [
            hit_rate_avg, precision_avg, recall_avg, f1_avg, resultados['n_evaluados'],
            ingreso_acertado_promedio, ingreso_total_promedio, tasa_captura_promedio,
            ingreso_total_acertado, ingreso_total_clientes, uplift_monetario
        ]
    }, index=[
        'hit_rate', 'precision', 'recall', 'f1_score', 'n_evaluaciones',
        'ingreso_promedio_acertado', 'ingreso_promedio_total', 'tasa_captura_promedio',
        'ingreso_total_acertado', 'ingreso_total_clientes', 'uplift_monetario_pct'
    ])

    

else:
    print("No se pudieron evaluar clientes")


MÉTRICAS TRADICIONALES:
  Hit Rate@10:     0.251 (25.1% clientes con al menos 1 acierto)
  Precision@10:    0.106
  Recall@10:       0.105
  F1-Score:        0.105
  Clientes evaluados: 11327

💰 MÉTRICAS DE INGRESOS:
  Ingreso promedio capturado por cliente: $8.86
  Ingreso promedio total por cliente: $119.79
  Tasa promedio de captura de ingresos: 7.18%
  Ingreso potencial promedio por recomendaciones: $49.71

💵 INGRESOS AGREGADOS:
  Total capturado por recomendaciones: $100,369.68
  Total gastado por clientes evaluados: $1,356,892.07
  Total potencial de recomendaciones: $563,052.54
  % del gasto total capturado: 7.40%

📊 ANÁLISIS DE ROI DEL ALGORITMO:
  Uplift monetario directo: 7.40%
  Ingresos adicionales por cada $100 gastados: $7.40
  Proyección de ingresos adicionales anuales por cliente: $106.33

DISTRIBUCIÓN DE RESULTADOS:
  Clientes sin aciertos: 8485 (74.9%)
  Clientes con aciertos: 2842 (25.1%)


In [13]:
# ================================================================
# COMPARACIÓN CON BASELINE ALEATORIO
# ================================================================

def evaluar_baseline_aleatorio(n_clientes_sample=200, k=10):
    """
    Evalúa un baseline aleatorio para comparar con el algoritmo histórico
    """
    print(f"\nEVALUANDO BASELINE ALEATORIO")
    print("="*40)
    
    # Obtener todos los productos disponibles en el modelo
    productos_disponibles = list(modelo_historico_importado['idx_to_item'].values())
    
    hit_rates_random = []
    precisions_random = []
    recalls_random = []
    
    # Evaluar muestra de clientes con recomendaciones aleatorias
    clientes_sample = np.random.choice(clientes_con_historial, min(n_clientes_sample, len(clientes_con_historial)), replace=False)
    
    for cliente_id in clientes_sample:
        # Generar recomendaciones aleatorias
        productos_aleatorios = set(np.random.choice(productos_disponibles, k, replace=False))
        
        # Productos realmente comprados
        productos_comprados = compras_reales_por_cliente[cliente_id]
        
        # Calcular métricas
        aciertos = productos_aleatorios.intersection(productos_comprados)
        hit_rate = 1 if len(aciertos) > 0 else 0
        precision = len(aciertos) / len(productos_aleatorios) if productos_aleatorios else 0
        recall = len(aciertos) / len(productos_comprados) if productos_comprados else 0
        
        hit_rates_random.append(hit_rate)
        precisions_random.append(precision)
        recalls_random.append(recall)
    
    # Calcular promedios
    hit_rate_random_avg = np.mean(hit_rates_random)
    precision_random_avg = np.mean(precisions_random)
    recall_random_avg = np.mean(recalls_random)
    
    print(f"BASELINE ALEATORIO (n={len(clientes_sample)}):")
    print(f"  Hit Rate@{k}: {hit_rate_random_avg:.3f}")
    print(f"  Precision@{k}: {precision_random_avg:.3f}")
    print(f"  Recall@{k}: {recall_random_avg:.3f}")
    
    # Comparar con algoritmo histórico
    if resultados['n_evaluados'] > 0:
        print(f"\nMEJORA vs BASELINE ALEATORIO:")
        mejora_hit_rate = (hit_rate_avg / hit_rate_random_avg - 1) * 100 if hit_rate_random_avg > 0 else float('inf')
        mejora_precision = (precision_avg / precision_random_avg - 1) * 100 if precision_random_avg > 0 else float('inf')
        
        print(f"  Hit Rate: {mejora_hit_rate:.1f}% mejor")
        print(f"  Precision: {mejora_precision:.1f}% mejor")
    
    return {
        'hit_rate': hit_rate_random_avg,
        'precision': precision_random_avg,
        'recall': recall_random_avg
    }

# Ejecutar comparación con baseline
baseline_results = evaluar_baseline_aleatorio(n_clientes_sample=len(clientes_con_historial), k=k_recomendaciones)


EVALUANDO BASELINE ALEATORIO
BASELINE ALEATORIO (n=11330):
  Hit Rate@10: 0.005
  Precision@10: 0.001
  Recall@10: 0.002

MEJORA vs BASELINE ALEATORIO:
  Hit Rate: 4718.2% mejor
  Precision: 20280.7% mejor
